In [ ]:
import torch
import torch.nn as nn
import sys
import copy
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from core.util.io import read_csv
from core.util.get_datasets import get_trefor_park_as_tensor

In [ ]:
batch_size = 64
learning_rate = 0.005
EPOCHS = 250
lookback = 24
loss_function = nn.HuberLoss()

In [ ]:
data_train = read_csv("processed/park_training.csv")
data_val = read_csv("processed/park_validation.csv")
data_test = read_csv("processed/park_testing.csv")

In [ ]:
x_train, y_train = get_trefor_park_as_tensor(data_train)
x_val, y_val = get_trefor_park_as_tensor(data_val)
x_test, y_test = get_trefor_park_as_tensor(data_test)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class TreforData(Dataset):
    """Initialize Trefor dataset."""

    def __init__(self, x: torch.tensor, y: torch.tensor) -> None:
        """Initialize dataset.

        Arguments:
            x: feature as torch
            y: target as torch

        """
        self.x = x.to(device)
        self.y = y.to(device)

    def __len__(self) -> int:
        """Return length of dataset."""
        return len(self.x)

    def __getitem__(self, i: int) -> tuple:
        """Return tuple from dataset."""
        return self.x[i], self.y[i]


train_dataset = TreforData(x_train, y_train)
val_dataset = TreforData(x_val, y_val)
test_dataset = TreforData(x_test, y_test)

In [ ]:
training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testing_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class SimpleGRU(nn.Module):
    """TODO."""

    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        num_layers: int,
    ) -> None:
        """TODO."""
        super(SimpleGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size

        self.gru = nn.GRU(
            input_size, hidden_size, num_layers=num_layers, batch_first=True
        )
        self.fc1 = nn.Linear(hidden_size * 24, 24)

    def forward(self, x: torch.tensor) -> torch.tensor:
        """TODO."""
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc1(out)
        return out


model = SimpleGRU(1, 5, 1)
model.to(device)
model = nn.DataParallel(model)
# model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
plot_train_loss = []
plot_val_loss = []


def train_one_epoch() -> float:
    """Train one epoch."""
    running_loss = 0.0
    last_loss = 0.0

    for i, data in enumerate(training_loader):
        # Every data instance is an input + target
        inputs, target = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        predictions = model(inputs)

        # Compute the loss and its gradient
        target = target.squeeze(-1)
        loss = loss_function(predictions, target)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 100 == 99:
            last_loss = running_loss / 100  # loss per 100 batch
            # print(f'  batch {i+1} loss: {last_loss}')
            running_loss = 0.0
    return last_loss

In [ ]:
epoch_number = 0
best_v_loss = sys.maxsize
best_model = None


for epoch in range(EPOCHS):
    print(f"EPOCH {epoch_number + 1}:")

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch()

    running_v_loss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, v_data in enumerate(validation_loader):
            v_inputs, v_target = v_data
            v_predictions = model(v_inputs)
            v_target = v_target.squeeze(-1)
            v_loss = loss_function(v_predictions, v_target)
            running_v_loss += v_loss.item()

    avg_v_loss = running_v_loss / (i + 1)

    # Log the running loss averaged per batch
    # for both training and validation
    # print(f'Training : {avg_loss}, Validation : {avg_v_loss}')
    plot_train_loss.append(avg_loss)
    plot_val_loss.append(avg_v_loss)

    # Track best performance, and save the model's state
    if avg_v_loss < best_v_loss:
        best_v_loss = avg_v_loss
        best_model = copy.deepcopy(model)

    epoch_number += 1

In [ ]:
best_model.eval()
predicted = []
t_loss = 0
num_batches = len(testing_loader)
size = len(testing_loader.dataset)
with torch.no_grad():
    for i, t_data in enumerate(testing_loader):
        t_inputs, t_target = t_data
        t_predictions = best_model(t_inputs)
        predicted.append(t_predictions)
        t_loss += loss_function(t_predictions, t_target).item()

predicted = torch.cat(predicted, dim=0)
t_loss /= num_batches
print(f"Avg loss: {t_loss:>8f} \n")

In [ ]:
plt.clf()
plt.plot(plot_train_loss, label="Training Loss")
plt.plot(plot_val_loss, label="Validation Loss")
plt.legend()
plt.show()

In [ ]:
# denormalized_y_test = scaler.inverse_transform(y_test.numpy())
# denormalized_predicted = scaler.inverse_transform(predicted.numpy())
plt.plot(y_test, label="Actual Consumption")
plt.plot(predicted, label="Predicted Consumption")
plt.xlabel("Hour")
plt.ylabel("Consumption")
plt.legend()
plt.show()